In [ ]:
using Logging
logger = ConsoleLogger(stdout)
# debuglogger = ConsoleLogger(stderr, Logging.Debug)
global_logger(logger)
using JLD

using Random
rng = MersenneTwister(1234)
import Dates

In [ ]:
using Plots
pyplot()

In [ ]:
using ForwardDiff
using ProgressMeter
using LinearAlgebra: dot
include("../utils/misc.jl")
using .MiscUtils: myCircle, mySphere

In [ ]:
ts = Dates.now()

## model parameters
input_dim = 2 # input dim
bias = true
m = 50 # nb hidden neurons for each sign
N = 5 # nb samples
activation_str = "ReLUcub"

## logging
ts_fsfriendly = Dates.format(ts, "yyyy-mm-ddTHHMMSS") # filesystem-friendly string for ts
resultdir = mkpath("../results/maxF1margin__dim$(input_dim)bias$(bias)__m$(m)__N$(N)__$(activation_str)__$(ts_fsfriendly)")

logfile = "$resultdir/log.txt"
touch(logfile)
open(logfile, "a") do f
    write(f, "input_dim=$input_dim\n")
    write(f, "bias: $bias\n")
    write(f, "m=$m\n")
    write(f, "N=$N\n")
    write(f, "activation: $activation_str\n")
end

In [ ]:
alpha = 8
scaling = 1 # unkown what the "correct" scaling typically is!

## algo parameters
T = Int(floor(4000 / (alpha^scaling)))
eta0_a = 1e-1 * alpha # initial stepsize
eta0_w = 1e-1 * alpha
eta0_theta = 1e-2 * alpha
constrain_theta = true
extrasteps = 2 # extrasteps=1: CP-MDA, extrasteps=2: CP-MP

open(logfile, "a") do f
    write(f, "T=$T\n")
    write(f, "eta0_a=$(eta0_a)\n")
    write(f, "eta0_w=$(eta0_w)\n")
    write(f, "eta0_theta=$(eta0_theta)\n")
    write(f, "constrain_theta: $(constrain_theta)\n")
    write(f, "extrasteps=$(extrasteps)\n")
end

In [ ]:
## plotting parameters
TNI_min = 1 # plot NI error
TNI_max = T
evalNIevery = Int(floor((TNI_max-TNI_min)/50))
Ntheta = Int(1e4)
skip_avg = true # avg not implemented

@assert input_dim==2
# r = range(-3, 3, length=101)
r = range(-2.2, 2.2, length=101)

Tplotreg_min = 1 # plot decision regions gif
Tplotreg_max = T
plotregevery = Int(floor((Tplotreg_max-Tplotreg_min)/50))
hidetitle = true
hidelabel = true
skip_gifs = true

In [ ]:
if activation_str == "ReLU"
    activation(x) = max(0, x)
elseif activation_str == "abs"
    activation(x) = abs(x)
elseif activation_str == "ReLUsq"
    activation(x) = max(0, x)^2
elseif activation_str == "ReLUcub"
    activation(x) = max(0, x)^3
elseif activation_str == "ReLUquar"
    activation(x) = max(0, x)^4
# elseif activation_str == "sq" # polynomial activation is silly
#     activation(x) = x^2
elseif activation_str == "sigmoid"
    activation(x) = 1/(1+exp(-x))
elseif activation_str == "tanh"
    activation(x) = tanh(x)
else
    error("activation_str not recognized, should be 
        \"ReLU\" or 
        \"abs\" or 
        \"ReLUsq\" or 
        \"ReLUcub\" or 
        \"ReLUquar\" or 
        \"sigmoid\" 
        or \"tanh\"")
end

if bias == true
    d = input_dim + 1
else
    d = input_dim
end

In [ ]:
function makedataset(d, N; bias, rng=MersenneTwister(1234))
    x = randn(rng, d, N)
    if bias
        x[d,:] .= 1
    end
    y = sign.(rand(rng, N).-0.5) # -1 or 1 uniformly
    return x, y
end

Random.seed!(rng, 1234)
x, y = makedataset(d, N; bias=bias, rng=rng)
save("$resultdir/dataset.jld", "x", x, "y", y)

scatter(x[1, y.==1], x[2, y.==1], m=:circ, markersize=8, label="+", color=:green)
scatter!(x[1, y.==-1], x[2, y.==-1], m=:utriangle, markersize=8, label="-", color=:red)

In [ ]:
##   max_nu min_i int_Theta yi*phi(theta, xi) dnu(theta) subj. to nu in M(Theta) and ||nu||=1
## = min_a max_nu sum_i int_{Theta+ U Theta-} ai yi*phitilde(theta, xi) dnu(theta) subj. to nu in P(Theta+ U Theta-) and a in P([N]), 
## where phitilde(theta, x) = phi(theta, x) if theta in Theta+ and -phi(theta, x) if theta in Theta- (cf Appdx A of Chizat21)
# the network is parametrized such that the first m neurons are positively weighted and the last m are negatively weighted
gfun(i, theta) = y[i] * activation(dot(theta, x[:,i]))

function logit(x, w, theta)
    out = 0
    for j=1:m
        out += w[j] * activation(dot(theta[:,j], x))
    end
    for j=m+1:2m
        out += -w[j] * activation(dot(theta[:,j], x))
    end
    return out
end

if bias
    predict(x1, x2; w, theta, hard=true) = hard ? sign(logit([x1, x2, 1], w, theta)) : logit([x1, x2, 1], w, theta)
else
    predict(x1, x2; w, theta, hard=true) = hard ? sign(logit([x1, x2], w, theta)) : logit([x1, x2], w, theta)
end

In [ ]:
Random.seed!(rng, 1234)
# initialize adversary weights
a = ones(N) ./ N
# initialize network weights and positions nu=(w, theta); theta constrained to unit l2 sphere
w = ones(2m) ./ (2m)
theta = randn(rng, d, 2m)
for j=1:2m
    theta[:,j] ./= sqrt(sum(theta[:,j].^2))
end

# for extragradient step
ap = similar(a)
wp = similar(w)
thetap = similar(theta)
# to store intermediate values
copies_a = Array{Float64}(undef, N, T+1)
copies_w = Array{Float64}(undef, 2m, T+1)
copies_theta = Array{Float64}(undef, d, 2m, T+1)
;

In [ ]:
## plot the decision region at random initialization https://discourse.julialang.org/t/plotting-decision-boundary-regions-for-classifier/21397
contour(r, r, 
    (x1, x2) -> predict(x1, x2; w=w, theta=theta, hard=false),
    f=true)

In [ ]:
"""
Take a CP gradient step
- starting from a, w, theta
- evaluating the gradients at ap, wp, thetap
- with stepsizes eta_a, eta_w, eta_theta
(taking care of the fact that the network is parametrized such that the first m neurons are positively weighted and the last m are negatively weighted)
Returns the updated particles a1, w1, theta1
"""
function step_CPMDA(
        f,
        a, w, theta,
        ap, wp, thetap,
        eta_a, eta_w, eta_theta;
        true_prox=false,
        constrain_theta=true
)
    N = length(a)
    d = size(theta)[1]
    m = Int(size(theta)[2] / 2)
    
    neuronsigns = ones(2m)
    neuronsigns[m+1:2m] .= -1

    Dfp = Array{Float64}(undef, d, N, 2m) # gradient of f w.r.t theta at thetap
    for i=1:N, j=1:2m
        Dfp[:,i,j] = ForwardDiff.gradient(tt -> f(i,tt), thetap[:,j])
    end

    # take step: adversary
    s = Array{Float64}(undef, N)
    for i=1:N
        s[i] = sum( neuronsigns[j] * wp[j] * f(i, thetap[:,j]) for j=1:2m )
    end
    if eta_a == Inf
        a1 = zeros(N)
        a1[argmax(s)] = 1.
    else
        a1 = a .* exp.(-eta_a * s)
        a1 ./= sum(a1)
    end
    
    # take step: network
    w1 = similar(w)
    theta1 = similar(theta)
    for j=1:2m
        s = sum( ap[i] * f(i, thetap[:,j]) for i=1:N )
        w1[j] = w[j] * exp(eta_w*neuronsigns[j]*s)
    end
    w1 ./= sum(w1)
    for j=1:2m
        s = sum( ap[i] * Dfp[:,i,j] for i=1:N )
        @assert size(s) == (d,)
        if true_prox
            theta1[:,j] = theta[:,j] + eta_theta * neuronsigns[j] * wp[j] / w[j] * s
        else
            theta1[:,j] = theta[:,j] + eta_theta * neuronsigns[j] * s
        end
    end
    # retract (just project) theta1 back to unit l2 sphere
    if constrain_theta
        for j=1:2m
            theta1[:,j] ./= sqrt(sum(theta1[:,j].^2))
        end
    end
    
    return a1, w1, theta1
end


In [ ]:
eta_a, eta_w, eta_theta = eta0_a, eta0_w, eta0_theta # constant stepsizes

@showprogress 1 for t=1:T # minimum update interval of 1 second
    copies_a[:,t] = copy(a)
    copies_w[:,t] = copy(w)
    copies_theta[:,:,t] = copy(theta)

    # extragradient ("ghost" step)
    # extrasteps=1: CP-MDA, extrasteps=2: CP-MP
    ap, wp, thetap = a, w, theta
    for s=1:extrasteps
        ap, wp, thetap = step_CPMDA(    
            gfun,
            a, w, theta,
            ap, wp, thetap,
            eta_a, eta_w, eta_theta;
            true_prox=true,
            constrain_theta=constrain_theta
        )
    end

    # take step
    a, w, theta = ap, wp, thetap
end

copies_a[:,T+1] = copy(a)
copies_w[:,T+1] = copy(w)
copies_theta[:,:,T+1] = copy(theta)

af, wf, thetaf = a, w, theta
save("$resultdir/iterates.jld", "copies_a", copies_a, "copies_w", copies_w, "copies_theta", copies_theta)

In [ ]:
af

In [ ]:
## plot the NI error
"""
Compute the "global" Nikaido-Isoda (NI) error
    max_{a0, nu0} <a|F|nu0> - <a0|F|nu> = max_theta <a|F|delta_theta> - min_i <ei|F|nu>
"""
function glob_NI_err(gfun, a, w, theta; Ntheta=Int(1e4))
    N = length(a)
    d = size(theta)[1]
    m = Int(size(theta)[2] / 2)
    
    if d==2
        apcont = myCircle(Ntheta)
    elseif d==3
        apcont, Ntheta_new = mySphere(Ntheta)
    else
        error("glob_NI_err not implemented for d>3")
    end
    maxtheta = -Inf
    mintheta = +Inf
    for k=1:Ntheta_new
        theta0 = apcont[:,k]
        s = 0
        for i=1:N
            s += a[i] * gfun(i, theta0)
        end
        maxtheta = max(maxtheta, s)
        mintheta = min(mintheta, s)
    end
    Fnu(i) = sum( w[j] * gfun(i, theta[:,j]) for j=1:m ) - sum( w[j] * gfun(i, theta[:,j]) for j=m+1:2m )
    mini = minimum(Fnu(i) for i=1:N)
    return max(maxtheta, -mintheta) - mini
end

In [ ]:
TNI_max = 400
evalNIevery = 1

In [ ]:
nierrs = Array{Float64}(undef, T+2)
@showprogress 1 for t=TNI_min:evalNIevery:TNI_max+1 # minimum update interval of 1 second
# for t=TNI_min:evalNIevery:TNI_max+1
    nierrs[t] = glob_NI_err(gfun, copies_a[:,t], copies_w[:,t], copies_theta[:,:,t]; Ntheta=Ntheta)
end
if !skip_avg
    nierrs[T+2] = glob_NI_err(gfun, avg_a, avg_w, avg_theta; Ntheta=Ntheta)
end
open(logfile, "a") do f
    for t=TNI_min:evalNIevery:TNI_max+1
        write(f, "glob_NI_err at iteration#$t: $(nierrs[t])\n")
    end
    if !skip_avg 
        write(f, "glob_NI_err at avg iterate: $(nierrs[T+2])\n") 
    end
end

save("$resultdir/nierrs__every$(evalNIevery)__t=$(TNI_min)--$(TNI_max).jld", "nierrs", nierrs)

plt_NI = plot(range(TNI_min, stop=TNI_max+1, step=evalNIevery), nierrs[TNI_min:evalNIevery:TNI_max+1], xlabel="k", label="")
if !skip_avg
    hline!([nierrs[T+2]], label="avg iterate")
end
if !hidetitle
    title!("NI error of iterates")
end
fn = "$resultdir/NI_errors.png"
savefig(plt_NI, fn)

In [ ]:
eps = 1e-10 # numerical stability (we use approximations (with deltax, deltay) to compute glob_NI_err)
upscale=2   # adjusting fontsize, linewidths etc.
resscale=2  # keep everything else fixed but increase resolution
fontsize=11/upscale*1.75

plt_NI_log = plot(range(TNI_min, stop=TNI_max+1, step=evalNIevery), eps .+ max.(0, nierrs[TNI_min:evalNIevery:(TNI_max+1)]), xlabel="k", label="", yscale=:log10,
    linewidth=upscale,
    xtickfontsize=fontsize, ytickfontsize=fontsize, xguidefontsize=fontsize, yguidefontsize=fontsize, legendfontsize=fontsize,
    dpi=upscale*100*resscale,
    size=(600/upscale, 400/upscale))
if !skip_avg
    hline!([ eps + max(0, nierrs[T+2]) ], label=(hidelabels ? "" : "avg iterate"))
end
if !hidetitle
    title!("NI error of iterates (log-linear scale)")
end
fn = "$resultdir/NI_errors_logscale.png"
savefig(plt_NI_log, fn)

plt_NI_log

In [ ]:
## decision region at the last iterate

upscale=2   # adjusting fontsize, linewidths etc.
resscale=2  # keep everything else fixed but increase resolution
fontsize=11/upscale*1.75

pltt = contour(r, r,
    (x1, x2) -> predict(x1, x2; w=w, theta=theta, hard=false),
    f=true,
    levels=-6:1.:6,
    clims=(-6, 6),
    linewidth=upscale,
    xtickfontsize=fontsize, ytickfontsize=fontsize, xguidefontsize=fontsize, yguidefontsize=fontsize, legendfontsize=fontsize*0.8,
    dpi=upscale*100*resscale,
    size=(600/upscale, 500/upscale))
contour!(r, r,
    (x1, x2) -> predict(x1, x2; w=w, theta=theta, hard=false),
    levels=[0.],
    seriescolor=:blues,
    linestyle=:dash,
    linewidth=3/upscale)
scatter!(x[1, y.==1], x[2, y.==1], m=:circ, markersize=8/upscale, markerstrokewidth=1/upscale, label="+", color=:green)  
scatter!(x[1, y.==-1], x[2, y.==-1], m=:utriangle, markersize=8/upscale, label=" -", markerstrokewidth=1/upscale, color=:red)

fn = "$resultdir/contour_soft_lastiter.png"
savefig(pltt, fn)
pltt

In [ ]:
using Plots.PlotMeasures # enable setting "margins=10mm" in plots
using LaTeXStrings

In [ ]:
upscale=2   # adjusting fontsize, linewidths etc.
resscale=2  # keep everything else fixed but increase resolution
fontsize=11/upscale*1.4

azimuth = 25
elevation = 55

phi_s = range(0, 2pi, length=100)
theta_s = range(0, pi, length=100)
r0 = 1/m
xss = r0 .* [cos(t)*sin(p) for t in theta_s, p in phi_s]
yss = r0 .* [sin(t)*sin(p) for t in theta_s, p in phi_s]
zss = r0 .* [cos(p) for t in theta_s, p in phi_s]
pltt = plot(xss, yss, zss, linetype=:surface, colorbar=false, seriescolor=:greens, alpha=0.1, 
    xlabel=L"θ_1", ylabel=L"θ_2", zlabel=L"θ_3",
    xlim=(-0.05, 0.05),
    ylim=(-0.05, 0.05),
    zlim=(-0.05, 0.05),
    camera = (azimuth, elevation),
    margins=10mm,
    xtickfontsize=0.6*fontsize, ytickfontsize=0.6*fontsize, ztickfontsize=0.6*fontsize,
    xguidefontsize=fontsize, yguidefontsize=fontsize, zguidefontsize=fontsize,
    legendfontsize=fontsize*0.8,
    dpi=upscale*100*resscale,
    size=(600/upscale, 600/upscale),
    aspect_ratio=:equal)
theta = thetaf
neur = zeros(3, 2m)
for j=1:2m
    neur[:,j] = w[j] * theta[:,j]
end
for j=1:m
    plot!([0, neur[1,j]], [0, neur[2,j]], [0, neur[3,j]], color=:red, label=false)
end
for j=m+1:2m
    plot!([0, neur[1,j]], [0, neur[2,j]], [0, neur[3,j]], color=:blue, label=false)
end
scatter!(neur[1,1:m],    neur[2,1:m],    neur[3,1:m]    , markersize=4, markercolor=:red, label=false)
scatter!(neur[1,m+1:2m], neur[2,m+1:2m], neur[3,m+1:2m] , markersize=4, markercolor=:blue, label=false)

fn = "$resultdir/neurons_lastiter_3D__azim$(azimuth)elev$(elevation).png"
savefig(pltt, fn)
pltt